In [1]:
import pickle
import numpy as np
import scipy
from scipy.spatial import distance

In [2]:
# Define varaibles

KB1 = "dbp"
KB2 = "wd"
SEED_RATIO = "30"
SEED_FOLDER = "0_"+SEED_RATIO[:1]
DATASET = KB1.upper()+"_"+KB2.upper()
K = 10

In [3]:
# Load Vocab

def get_vocab(filename):
    fo = open(filename,"r")
    list_id = list()
    id2ent=dict()
    ent2id=dict()
    counter = 0
    for line in fo:
        line_split = line.split()
        the_id = counter
        the_ent = line_split[1]
        id2ent[int(the_id)] = the_ent
        ent2id[the_ent] = int(the_id)
        list_id.append(int(line_split[0]))
        counter+=1
    fo.close()
    return id2ent, ent2id, list_id

kb1_vocab_filename = './data/'+KB2+'/'+SEED_FOLDER+'/'+'ent_ids_1'
kb2_vocab_filename = './data/'+KB2+'/'+SEED_FOLDER+'/'+'ent_ids_2'

kb1_id2ent, kb1_ent2id, kb1_ids = get_vocab(kb1_vocab_filename)
kb2_id2ent, kb2_ent2id, kb2_ids = get_vocab(kb2_vocab_filename)

In [4]:
# Load Embeddings

all_emb_filename = './data/'+KB2+'/'+SEED_FOLDER+'/SE_'+KB2+'_'+KB1
all_emb = pickle.load(open(all_emb_filename, 'rb'))

se = all_emb
se_kb1_emb = np.take(se, kb1_ids, axis=0)
se_kb2_emb = np.take(all_emb, kb2_ids, axis=0)

In [ ]:
fmap = open('./data/'+KB2+'/'+SEED_FOLDER+'/'+'test_ent_ids', 'r')

confidence_list = list()
for line in fmap:
    line_split = line.rstrip().split('\t')
    src_ent = int(line_split[0])
    tar_ent = int(line_split[1])-len(se_kb1_emb)
    
    src_emb = se_kb1_emb[src_ent,:].reshape(1,-1)
    sim = scipy.spatial.distance.cdist(src_emb, se_kb2_emb, metric='cosine') #change metric here
    
    top_k = sim[0].argsort()[:K]
    
    if(tar_ent == top_k[0]):
        confidence_list.append(sim[0][top_k[1]] - sim[0][top_k[0]]) # confidence -> second best minus top 

print (np.mean(confidence_list))
fmap.close()
